# Loading The Required Librariries

Using these libraries as:

- tweepy for reading tweets
- pandas for stroring data
- for JSON

In [ ]:
import tweepy
import pandas as pd


# Initialising The Credentials

Defining the credentials used for extracting the tweets

In [2]:
Consumer_Key = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX" 
Consumer_Key_Secret = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
Access_Key = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
Access_Key_Secret = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

# Function to Extract Tweets

In [ ]:
def get_tweets(username): 
          
        auth = tweepy.OAuthHandler(Consumer_key, Consumer_secret) 
        auth.set_access_token(Access_key, Access_secret) 
        api = tweepy.API(auth) 
  
        tweets = api.user_timeline(screen_name = username,count=200)
        twts=[]  

        for j in tweets: 
  
            twts.append(j)  
  
        print(twts) 

In [ ]:
get_tweets("midasIIItd")  